In [62]:
from mxnet.gluon import nn
import mxnet as mx
    

In [63]:
mx.__version__

'1.1.0'

In [64]:
def vgg_block(num_convs,channel ):
    out = nn.Sequential()
    for _ in range(num_convs):
        out.add(
            nn.Conv2D(channels=channel,kernel_size=3,padding=1,activation="relu"))
    out.add(nn.MaxPool2D(pool_size=2,strides=2))
    return out

In [65]:
block = vgg_block(2,128)
block.initialize()
x = mx.ndarray.random.uniform(shape=(2,3,16,16))
y = block(x)
print(y.shape)
block.collect_params()

(2, 128, 8, 8)


sequential0_ (
  Parameter conv0_weight (shape=(128, 3, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter conv0_bias (shape=(128,), dtype=<class 'numpy.float32'>)
  Parameter conv1_weight (shape=(128, 128, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter conv1_bias (shape=(128,), dtype=<class 'numpy.float32'>)
)

In [66]:
def vgg_stack(layer):
    out = nn.Sequential()
    for num_convs,channels in layer:
        out.add(vgg_block(num_convs,channels))
    return out

In [67]:
layer = ((1,64),(1,128),(2,256),(2,152),(2,512))
out = nn.Sequential()
with out.name_scope():
    out.add(
        vgg_stack(layer))
    out.add(
        #fc1
        nn.Flatten(),
        nn.Dense(4096,activation='relu'),
        nn.Dropout(.5),
        #fc2
        nn.Dense(4096,activation='relu'),
        nn.Dropout(.5),
        #fc3
        nn.Dense(10),
    )
out

Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Conv2D(None -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2D(None -> 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2D(None -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Conv2D(None -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    )
    (3): Sequential(
      (0): Conv2D(None -> 152, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Conv2D(None -> 152, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    )
    (4): Sequential(
      (0): Conv2D(Non

In [68]:
def vgg16():
    """use vgg16 -D network"""
    layer = ((2,64),(2,128),(3,256),(3,512),(3,512))
    out = nn.Sequential()
    with out.name_scope():
        out.add(vgg_stack(layer))
        out.add(
            nn.Flatten(),
            nn.Dense(4096,activation="relu"),
            nn.Dropout(.5),
            nn.Dense(4096,activation="relu"),
            nn.Dropout(.5),
            nn.Dense(10),
        )
    return out
    

In [69]:
out1 = vgg16()
x = mx.ndarray.random.uniform(shape=(1,3,224,224))
out1.initialize()
y = out1(x)
print(y)
out1.collect_params()


[[  11.40312386  -86.82865906  -70.86930847  -35.04047775   79.72298431
   118.66650391   34.95783997   23.84843063 -118.40135956  -18.04834175]]
<NDArray 1x10 @cpu(0)>


sequential2_ (
  Parameter sequential2_conv0_weight (shape=(64, 3, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter sequential2_conv0_bias (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter sequential2_conv1_weight (shape=(64, 64, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter sequential2_conv1_bias (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter sequential2_conv2_weight (shape=(128, 64, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter sequential2_conv2_bias (shape=(128,), dtype=<class 'numpy.float32'>)
  Parameter sequential2_conv3_weight (shape=(128, 128, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter sequential2_conv3_bias (shape=(128,), dtype=<class 'numpy.float32'>)
  Parameter sequential2_conv4_weight (shape=(256, 128, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter sequential2_conv4_bias (shape=(256,), dtype=<class 'numpy.float32'>)
  Parameter sequential2_conv5_weight (shape=(256, 256, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter sequential2_conv5_

In [141]:
out1

Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Conv2D(3 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2D(64 -> 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Conv2D(128 -> 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2D(128 -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Conv2D(256 -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): Conv2D(256 -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    )
    (3): Sequential(
      (0): Conv2D(256 -> 512, kernel_size=(3, 3), stride=(1,

In [142]:

params = out1.collect_params()
p =[]
layerNum = 0
sum1=0
for param in params.keys():
    values = params.get(param[13:]).shape
    sum0 = 1
    if layerNum%2==0:
        for i,value in enumerate(values):
            print(value)
            sum0 *=value
        sum1 += sum0
    else:
        sum1 += values[0]
    
sum1*4/(1024*1024)

TypeError: 'NoneType' object is not iterable

In [143]:
p[0][13:]

IndexError: list index out of range

In [144]:
import utils1
from mxnet import  gluon
from mxnet import init

In [145]:
net = nn.Sequential()
# add name_scope on the outer most Sequential
with net.name_scope():
    net.add(
        mlpconv(96, 11, 0, strides=4),
        mlpconv(256, 5, 2),
        mlpconv(384, 3, 1),
        nn.Dropout(.5),
        # 目标类为10类
        mlpconv(10, 3, 1, max_pooling=False),
        # 输入为 batch_size x 10 x 5 x 5, 通过AvgPool2D转成
        # batch_size x 10 x 1 x 1。
        # 我们可以使用 nn.AvgPool2D(pool_size=5),
        # 但更方便是使用全局池化，可以避免估算pool_size大小
        nn.GlobalAvgPool2D(),
        # 转成 batch_size x 10
        nn.Flatten()
    )

NameError: name 'mlpconv' is not defined

In [ ]:
train_data,test_data = utils1.load_data_fashion_mnist(batch_size=64,resize=224)
ctx = utils1.try_gpu()
out.initialize(ctx=ctx, init=init.Xavier())
loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(out.collect_params(),'sgd',{'learning_rate':0.05})
ctx

/home/wk/anaconda3/envs/gluon/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter sequential1_conv0_bias is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/home/wk/anaconda3/envs/gluon/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter sequential1_conv1_bias is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/home/wk/anaconda3/envs/gluon/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter sequential1_conv2_bias is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/home/wk/anaconda3/envs/gluon/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter sequential1_conv3_bias is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "

gpu(0)

In [ ]:
utils1.train(train_data,test_data,out,loss,trainer,ctx=ctx,num_epochs=1)

Start training on  gpu(0)


In [ ]:
def w1(func1):
    def inner(a,b):
        print('inner function\n')
        s = func1(a,b)
        return s
#         return fuc1()
    print('w1 function\n')
    return inner


In [ ]:
@w1
def fuc(a,b):
    print('111\n')
#     return a+b

In [ ]:
# w1(fuc)
fuc(3,2)

### 增加kernel=（1，1）共享权重来替代fc层

In [128]:
def mplconv(channels,kernel,padding,stride=1,pool_layer=True):
    net = nn.Sequential()
    with net.name_scope():
        net.add(
            nn.Conv2D(channels,kernel_size=kernel,strides=stride,padding=padding,activation="relu"),
            nn.Conv2D(channels,kernel_size=1,activation="relu"),
            nn.Conv2D(channels,kernel_size=1,activation="relu"),
        )
        if pool_layer:
            net.add(nn.MaxPool2D(pool_size=(3,3),strides=2,ceil_mode=False))
        return net

In [129]:
x = mx.ndarray.random.uniform(shape=(32,3,16,16))
net = mplconv(64,3,0)
net.initialize()
y = net(x)
y.shape
# net.collect_params()
# net

(32, 64, 6, 6)

In [130]:
net1 = nn.Sequential()
with net1.name_scope():
    net1.add(
        mplconv(96,11,0,stride=4),
        mplconv(256,5,2),
        mplconv(384,3,1),
        nn.Dropout(.5),
        mplconv(10,3,1),
        nn.GlobalAvgPool2D(),
        nn.Flatten()
    )

In [131]:
net1.collect_params()


sequential21_ (
  Parameter sequential21_sequential0_conv0_weight (shape=(96, 0, 11, 11), dtype=<class 'numpy.float32'>)
  Parameter sequential21_sequential0_conv0_bias (shape=(96,), dtype=<class 'numpy.float32'>)
  Parameter sequential21_sequential0_conv1_weight (shape=(96, 0, 1, 1), dtype=<class 'numpy.float32'>)
  Parameter sequential21_sequential0_conv1_bias (shape=(96,), dtype=<class 'numpy.float32'>)
  Parameter sequential21_sequential0_conv2_weight (shape=(96, 0, 1, 1), dtype=<class 'numpy.float32'>)
  Parameter sequential21_sequential0_conv2_bias (shape=(96,), dtype=<class 'numpy.float32'>)
  Parameter sequential21_sequential1_conv0_weight (shape=(256, 0, 5, 5), dtype=<class 'numpy.float32'>)
  Parameter sequential21_sequential1_conv0_bias (shape=(256,), dtype=<class 'numpy.float32'>)
  Parameter sequential21_sequential1_conv1_weight (shape=(256, 0, 1, 1), dtype=<class 'numpy.float32'>)
  Parameter sequential21_sequential1_conv1_bias (shape=(256,), dtype=<class 'numpy.float32'>

In [1]:
import utils1
import mxnet
from mxnet import  init
from mxnet import gluon
train_data, test_data = utils1.load_data_fashion_mnist(
    batch_size=64, resize=96)

ctx = utils1.try_gpu()
print(ctx)
net1.initialize(ctx=mxnet.gpu, init=init.Xavier())

loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),
                        'sgd', {'learning_rate': 0.1})


cpu(0)


NameError: name 'net1' is not defined

In [120]:
utils1.train(train_data, test_data, net, loss,
            trainer, ctx, num_epochs=1)

Start training on  gpu(0)


RuntimeError: Parameter sequential17_conv0_weight was not initialized on context gpu(0). It was only initialized on [cpu(0)].